title
purpose

In [4]:
import os,csv,json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import tifffile as tif
from scipy.spatial import distance

In [5]:
trainingfolder = '/home/emilyjanedennis/Desktop/training_inputs'
rawfolder = os.path.join(trainingfolder,"validate_raw")
# for each file want all the associated files
# either need to use the index or use the base name


In [6]:
# only have to do this once
# make rois into np arrays
roifolder = os.path.join(trainingfolder,"rois")
roilist = os.listdir(roifolder)

for folder in roilist:
    filesinfolder = os.listdir(os.path.join(roifolder,folder))
    rois=[]
    for element in range(0,np.size(filesinfolder)):
        fullnm = filesinfolder[element]
        z=int(fullnm[0:4])
        y=int(fullnm[5:9])
        x=int(fullnm[10:14])
        rois.append([z,y,x])
    np.save(os.path.join(rawfolder,"rois_as_npy/{}_rois.npy".format(folder[0:-6])),rois)
    
    

In [8]:
listoffiles = os.listdir(rawfolder)
listoffiles = ['z269stackstart150_ejd_cells_raw.npy',
 'z269stackstart100_ejd_cells_raw.npy',
 'z266stackstart400_set2_cells_raw.npy',
 'z269stackstart400_ejd_cells_raw.npy',
 'z266stackstart400_set2_ejd_cells_raw.npy']
numfiles = np.size(listoffiles)


In [9]:
listoffiles

['z269stackstart150_ejd_cells_raw.npy',
 'z269stackstart100_ejd_cells_raw.npy',
 'z266stackstart400_set2_cells_raw.npy',
 'z269stackstart400_ejd_cells_raw.npy',
 'z266stackstart400_set2_ejd_cells_raw.npy']

In [513]:
alldf = pd.DataFrame(columns=['name','sz1','sz2','source','tp','fp','fn','p','r','f1'])

for file in os.listdir(rawfolder):
    rawdata = np.load(os.path.join(rawfolder,file))
    xs =[]
    ys=[]
    zs=[]
    rawrois=[]
    for val in range(0,np.size(rawdata)-1):
        z=rawdata[val][0]
        y=rawdata[val][1]
        x=rawdata[val][2]
        rawrois.append([z,y,x])


        #load and reformat annotated rois
    rois=np.load(os.path.join(trainingfolder,"rois_as_numpy/{}_rois.npy".format(file[0:-14])))
    print(os.path.join(trainingfolder,"rois_as_numpy/{}_rois.npy".format(file[0:-14])))
    annotatedrois=[()]
    for l in range(0,np.shape(rois)[0]):
        vals = rois[l][0], rois[l][1], rois[l][2]
        annotatedrois.append(vals)
    anns = annotatedrois[1:]

    # set iters you want -- should move this out of loop and to the top 
    sizes = np.arange(10,100,5)
    sizes2 = np.arange(40,140,5)
    alldata=[()]
    # for this iter
    for sz in sizes:
        for sz2 in sizes2:
            size = (sz,sz2)
                #set thresholds
            thresholds = {    
                 'source' : 3,
                'size'   : size}
             #filter cells using defined below (need to import later, but had changed so source/sink = just sink)
            filtd = filter_cells(source = np.load(os.path.join(rawfolder,file)), 
                       thresholds=thresholds);
            filtereddata=[()]
            #reformat filtered data this is hacky and because I'm being lazy, to change
            for l in range(0,np.size(filtd)):
                z, y, x, toss, toss2 = filtd[l]
                vals = x, y, z
                filtereddata.append(vals)
            filt = filtereddata[1:]
            #compare filt to ann
            if filt:
                paired, tp, fp, fn = pairwise_distance_metrics(anns,filt,cutoff=30,verbose=False)
                # add output to other outputs
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1 = 2*((precision*recall)/(precision+recall))
                datatoadd = file[0:-14], sz, sz2, source, tp, fp, fn, round(precision,2), round(recall,2), round(f1,2)
                alldata.append(datatoadd)
    # when all loops are done, save
    placetosave = os.path.join(trainingfolder,"{}_filt_outputs.npy".format(file[0:-14]))
    np.save(placetosave,alldata)
    df = pd.DataFrame(alldata,columns=['name','sz1','sz2','source','tp','fp','fn','p','r','f1'])
    alldf = pd.concat([alldf,df])

/home/emilyjanedennis/Desktop/training_inputs/rois_as_numpy/z266stackstart250_ejd_rois.npy
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past 

/home/emilyjanedennis/anaconda3/envs/ClearMap/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted


got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted


got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted


got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
/home/emilyjanedennis/Desktop/training_inputs/rois_as_numpy/z266stackstart420_set2_ejd_rois.npy
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got 

got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted


got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted
got past zip
sorted


z269stackstart150_ejd
z269stackstart100_ejd
z266stackstart400_set2
z269stackstart400_ejd
z266stackstart400_set2_ejd


In [15]:
alldf.to_csv('/home/emilyjanedennis/Desktop/training_inputs/validate_raw/validation_20201002.csv')

ValueError: No objects to concatenate

In [13]:
alldata

NameError: name 'alldata' is not defined

In [3]:
def pairwise_distance_metrics_given_cdists(
        ground_truth, predicted, y, cutoff=10, verbose=True):
    """
    Function to calculate the pairwise distances
    between two lists of zyx points.

    Inputs:
    -------
    ground_truth, predicted: each iterable
    consisting of ndimensional coordinates.
    y: matrix of distances between all
    elements of ground truth and predicted

    Returns:
    -------
    paired: list of [ground_truth"s index
    (from input list), predicted"s index (from input list), distance]
    tp,fp,fn: statistics on true positives,
    false positives, and false negatives.
    """
    # only keep those distances that are below the cutoff!
    truth_indices, pred_indices = np.where(y <= cutoff)
    dists = zip(y[truth_indices, pred_indices], truth_indices, pred_indices)

    # sort by smallest dist
    dists = sorted(dists, key=lambda x: x[0])

    used_truth = set()
    used_pred = set()
    paired = []
    for (i, dist) in enumerate(dists):
        d = dist[0]
        if d > cutoff:
            # we have reached distances beyond the cutoff
            break
        truth_idx = dist[1]
        pred_idx = dist[2]
        if truth_idx not in used_truth and pred_idx not in used_pred:
            paired.append((truth_idx, pred_idx, d))
            used_truth.add(truth_idx)
            used_pred.add(pred_idx)
        if len(used_truth) == len(ground_truth) or len(used_pred) == len(predicted):
            # we have used up all the entries from the shorter list
            break

    tp = len(paired)
    fn = len(ground_truth) - len(paired)
    fp = len(predicted) - len(paired)
    if verbose:
        print("TP: {}, FP: {}, FN: {}".format(tp, fp, fn))
    # print(paired)

    if verbose:
       plt.hist([xx[2] for xx in paired], bins=np.max((int(len(paired)/500), 10)))
       plt.title("Histogram of distances - pixel or microns")

    return paired, tp, fp, fn

In [2]:
def filter_cells(source, thresholds):
  """Filter a array of detected cells according to the thresholds.
  
  Arguments
  ---------
  source : str, array or Source
    The source for the cell data.
  sink : str, array or Source
    The sink for the results.
  thresholds : dict
    Dictionary of the form {name : threshold} where name refers to the 
    column in the cell data and threshold can be None, a float 
    indicating a minimal threshold or a tuple (min,max) where min,max can be
    None or a minimal and maximal threshold value.
  
  Returns
  -------
  sink : str, array or Source
    The thresholded cell data.
  """
  
  ids = np.ones(source.shape[0], dtype=bool);
  for k,t in thresholds.items():
    if t:
      if not isinstance(t, (tuple, list)):
        t = (t, None);
      if t[0] is not None:
        ids = np.logical_and(ids, t[0] <= source[k])
      if t[1] is not None:
        ids = np.logical_and(ids, t[1] > source[k]);
  cells_filtered = source[ids];
  return cells_filtered


In [1]:
def pairwise_distance_metrics(ground_truth, predicted, cutoff=10, verbose=True):
    """
    Function to calculate the pairwise distances
    between two lists of zyx points.

    Inputs:
    -------
    ground_truth, predicted: each iterable consisting of
    ndimensional coordinates.

    Returns:
    -------
    paired: list of [ground_truth"s index (from input list),
    predicted"s index (from input list), distance]
    tp,fp,fn: statistics on true positives, false positives,
    and false negatives.
    """

    if verbose:
        print("\nCalculating pairwise distances...")
    y = distance.cdist(ground_truth, predicted, metric="euclidean")
    return pairwise_distance_metrics_given_cdists(
       ground_truth, predicted, y, cutoff, verbose)